---
#### LAZ File Reading, Writing and Plotting 

1. Read LAZ files from disk using PDAL and extract spatial boundary information.
2. Graph data and create visualizations of the point clouds using PYLAS.
---

## PDAL
#### Load pdal and other dependencies

https://pdal.io/python.html#python  
conda install python-pdal

In [ ]:
import pdal, json, os, subprocess
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.mlab as mlab
from matplotlib import cm
from descartes import PolygonPatch

from tkinter import Tk
from tkinter.filedialog import askopenfilename

#matplotlib.use('Agg')

In [ ]:
# It's always nice to check that you're in the right place.
%pwd
# The notebook usually opens in the notebook subfolder, so you may want to change to the base directory using the code below.
%cd ..

#### <b>Import</b> .laz file from your computer
Note: You will need to supply your own .laz file from a local directory. An example file (SNEX19_CRREL.laz) is included in the /data folder, but due to size limitations will need to be uploaded from your local machine.

In [ ]:
Tk().withdraw()
file_name = askopenfilename()
print(file_name)

#### <b>Read</b> point cloud data using the pdal.io <i> pipeline </i> method
Read more: https://pdal.io/pipeline.html

In [ ]:
# Load in the LAZ file
lasfile = file_name

pipeline = [
        {
            "type": "readers.las",
            "filename": lasfile
        }
    ]

pipeline = pdal.Pipeline(json.dumps(pipeline))
pipeline.execute()
arrays = pipeline.arrays

#### <b>Print</b> point cloud file hierarchy information 

In [ ]:
# Prints a point count of the data
print('Point count:', json.loads(pipeline.metadata)["metadata"]["readers.las"]["count"])

In [ ]:
# Prints the .las file schema, or hierarchy
pipeline.schema

#### <b>Print</b> a (very long) list of file metadata

In [ ]:
json.loads(pipeline.metadata)["metadata"]["readers.las"]

---
#### <b>Print</b> spatial information using <i>subprocess</i> package

In [ ]:
# This subprocess is necessary to extract -info, which contains spatial information using python
result = subprocess.run(['pdal', 'info', lasfile],
                        stderr = subprocess.PIPE,  # stderr and stdout get
                        stdout = subprocess.PIPE)  # captured as bytestrings

json_result = json.loads(result.stdout.decode())

In [ ]:
print('Converted to WGS84:')
json_result['stats']['bbox']['EPSG:4326']

In [ ]:
#Run this code if you want coorindates in native UTM coordinates
#json_result['stats']['bbox']['native']

In [ ]:
coords = json_result['stats']['bbox']['EPSG:4326']['boundary']['coordinates']
print(coords)

In [ ]:
poly = json_result['stats']['bbox']['EPSG:4326']['boundary']
print(poly)

In [ ]:
BLUE = '#6699cc'
fig = plt.figure() 
ax = fig.gca() 
ax.add_patch(PolygonPatch(poly, fc=BLUE, ec=BLUE, alpha=0.5, zorder=2 ))
ax.axis('scaled')
plt.show()

## PYLAS
### <b>Import and plot</b> data using the <i>pylas</i> package  
<b>*Note:</b> this method (for unkown reasons) distorts data values <i>BUT</i> will visualize the data correctly, so should only be used for plotting

In [ ]:
#pylas dependencies
#pip install pylas[lazrs]
import pylas
import lazrs
import s3fs
from laspy.file import File
import datashader as ds
import datashader.transfer_functions as tf

In [ ]:
# Command line option: !LAStools/bin/laszip -i /filepath/file.laz -o SNEX19_CRREL.las

In [ ]:
sample_data = './data/SNEX19_CRREL.laz'
export_path = 'export//'

In [ ]:
inFile = File(sample_data, mode='r')

In [ ]:
df = pd.DataFrame() 
df['X'] = inFile.X 
df['Y'] = inFile.Y 
df['Z'] = inFile.Z
df['intensity'] = inFile.intensity
display(df)

#### Create a simple plot using <i>matplotlib</i>

In [ ]:
plt.scatter(df.X, df.Y, c=df.intensity, marker='.', edgecolor='none')
plt.plot()

#### Create a larger plot using <i>datashader</i>

In [ ]:
cvs = ds.Canvas(plot_width=1000, plot_height=1000)
agg = cvs.points(df, 'X', 'Y', ds.mean('Z'))
img = tf.shade(agg)
tf.set_background(tf.shade(agg, cmap=cm.inferno),"black")

### <b>Plot and Save</b> a histogram of the LAZ data  
Get a visual summary of your data

In [ ]:
# Will require install of 
# def make_plot(dimensions, filename, dpi=300):
#     figure_position = 1
#     row = 1
#     fig = plt.figure(figure_position, figsize=(6, 8.5), dpi=dpi)
#     keys = dimensions.dtype.fields.keys()

#     for key in keys:
#         dimension = dimensions[key]
#         ax = fig.add_subplot(len(keys), 1, row)

#         n, bins, patches = ax.hist( dimension, 30,
#                                     #normed=0,
#                                     facecolor='grey',
#                                     alpha=0.75,
#                                     align='mid',
#                                     histtype='stepfilled',
#                                     linewidth=None)

#         ax.set_ylabel(key, size=10, rotation='horizontal')
#         ax.get_xaxis().set_visible(False)
#         ax.set_yticklabels('')
#         ax.set_yticks((),)
#         ax.set_xlim(min(dimension), max(dimension))
#         ax.set_ylim(min(n), max(n))
#         row = row + 1
#         figure_position = figure_position + 1
#     output = BytesIO()
#     plt.savefig(output,format="PNG")

#     o = open(filename, 'wb')
#     o.write(output.getvalue())
#     o.close()
#     return True

# make_plot(arrays[0], 'histogram.png')